In [1]:
import os

os.environ["CV_IO_MAX_IMAGE_PIXELS"] = f"{2**99}"
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = f"{2**63}"

import numpy as np
import matplotlib.pyplot as plt
import json
import cv2
import tiffslide

In [2]:
def get_binary_mask(wsi_path, annotations_path):
    with open(annotations_path) as f:
        annotations = json.load(f)
    # Load the WSI file
    slide = tiffslide.open_slide(wsi_path)
    dimensions = slide.dimensions
    width, height = dimensions
    
    binary_mask = np.zeros((height, width), dtype=np.uint8)
    if "annotation" in annotations:
        elements = annotations['annotation']['elements']
    else:
        elements = annotations[0]['elements']
    for element in elements:
        if element['type'] == 'polyline':
            points = element['points']
            # print(len(points[0]))
            if element['closed'] and len(points) > 0 and len(points[0]) == 3:
                points = np.array(points, dtype=np.int32)
                # Ensure the points are in the correct shape (N, 1, 2)
                points = np.array([[p[0], p[1]] for p in points], dtype=np.int32)
                points = points.reshape((-1, 1, 2))
                contours = [points]
                cv2.drawContours(binary_mask, contours, -1, color=255, thickness=cv2.FILLED)  # Draw the filled contour

    slide.close()
    # print('Done!')
    return binary_mask
    

In [ ]:


WSI_images_dir = "/blue/pinaki.sarder/manojkumargalla/PostProcess/data/WSIs/"
tbm_annotations_dir = '/blue/pinaki.sarder/manojkumargalla/PostProcess/data/TBM_Annotations/'
tubules_annotations_dir = '/blue/pinaki.sarder/manojkumargalla/PostProcess/data/Tubule_Annotations/'
filtered_wsi_dir = '/blue/pinaki.sarder/manojkumargalla/PostProcess/data/Filtered_WSI_images/'

WSI_images = os.listdir(WSI_images_dir)
print(WSI_images)


for wsi in WSI_images:
    wsi_path = os.path.join(WSI_images_dir, wsi)
    wsi_name = str(wsi.split('.')[0])
    print(f'Processing image - {wsi_name}')
    tbm_annotations_path = os.path.join(tbm_annotations_dir, f'{wsi_name}_tbm_jul18.json')
    tubules_annotations_path = os.path.join(tubules_annotations_dir, f'{wsi_name}.json')
    print('Generating binary masks')
    binary_mask_of_tubules = get_binary_mask(wsi_path, tubules_annotations_path)
    binary_mask_of_tbms = get_binary_mask(wsi_path, tbm_annotations_path)
    print('performing distancec transform')
    distance_transform = cv2.distanceTransform(binary_mask_of_tubules, cv2.DIST_L2, 5)
    inverse_binary_mask = 255 - binary_mask_of_tubules
    inverse_distance_transform = cv2.distanceTransform(inverse_binary_mask, cv2.DIST_L2, 5)
    distance_map = distance_transform - inverse_distance_transform
    threshold = 25
    mask = np.abs(distance_map) < threshold
    # Convert boolean mask to binary (0 and 255)
    threshInv = np.where(mask, 255, 0).astype(np.uint8)
    product = threshInv * binary_mask_of_tbms
    print(f'Saving filtered image - {wsi_name}')
    filtered_image_path = os.path.join(filtered_wsi_dir, f'{wsi_name}.png')
    cv2.imwrite(filtered_image_path, 255*product)
    print(f'Succesfully filtered image - {wsi_image}')
        
    

['S-2103-004857_PAS_2of2.svs', 'S-2106-003588_PAS_1of2.svs', 'S-2001-005357_PAS_1of2.svs', 'S-1904-007293_PAS_1of2.svs', 'S-1910-000089_PAS_2of2.svs', 'S-1905-018731_PAS_2of2.svs', '18-162_PAS_4of6.svs', 'S-1909-007149_PAS_1of2.svs', 'S-1905-017738_PAS_1of2.svs', 'S-1908-010066_PAS_1of2.svs']
Processing image - S-2103-004857_PAS_2of2
Generating binary masks
performing distancec transform


In [ ]:

from concurrent.futures import ThreadPoolExecutor

def process_image(wsi, WSI_images_dir, tbm_annotations_dir, tubules_annotations_dir, filtered_wsi_dir):
    wsi_path = os.path.join(WSI_images_dir, wsi)
    wsi_name = str(wsi.split('.')[0])
    print(f'Processing image - {wsi_name}')
    tbm_annotations_path = os.path.join(tbm_annotations_dir, f'{wsi_name}_tbm_jul18.json')
    tubules_annotations_path = os.path.join(tubules_annotations_dir, f'{wsi_name}.json')
    print('Generating binary masks')
    binary_mask_of_tubules = get_binary_mask(wsi_path, tubules_annotations_path)
    binary_mask_of_tbms = get_binary_mask(wsi_path, tbm_annotations_path)
    print('performing distance transform')
    distance_transform = cv2.distanceTransform(binary_mask_of_tubules, cv2.DIST_L2, 5)
    inverse_binary_mask = 255 - binary_mask_of_tubules
    inverse_distance_transform = cv2.distanceTransform(inverse_binary_mask, cv2.DIST_L2, 5)
    distance_map = distance_transform - inverse_distance_transform
    threshold = 25
    mask = np.abs(distance_map) < threshold
    # Convert boolean mask to binary (0 and 255)
    threshInv = np.where(mask, 255, 0).astype(np.uint8)
    product = threshInv * binary_mask_of_tbms
    print(f'Saving filtered image - {wsi_name}')
    filtered_image_path = os.path.join(filtered_wsi_dir, f'{wsi_name}.png')
    cv2.imwrite(filtered_image_path, 255 * product)
    print(f'Successfully filtered image - {wsi_name}')

def process_images():
    WSI_images_dir = "/blue/pinaki.sarder/manojkumargalla/PostProcess/data/WSIs/"
    tbm_annotations_dir = '/blue/pinaki.sarder/manojkumargalla/PostProcess/data/TBM_Annotations/'
    tubules_annotations_dir = '/blue/pinaki.sarder/manojkumargalla/PostProcess/data/Tubule_Annotations/'
    filtered_wsi_dir = '/blue/pinaki.sarder/manojkumargalla/PostProcess/data/Filtered_WSI_images/'

    WSI_images = os.listdir(WSI_images_dir)
    print(WSI_images)

    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(process_image, wsi, WSI_images_dir, tbm_annotations_dir, tubules_annotations_dir, filtered_wsi_dir) for wsi in WSI_images]
        for future in futures:
            future.result()  # This will raise exceptions if any

process_images()


['S-2103-004857_PAS_2of2.svs', 'S-2106-003588_PAS_1of2.svs', 'S-2001-005357_PAS_1of2.svs', 'S-1904-007293_PAS_1of2.svs', 'S-1910-000089_PAS_2of2.svs', 'S-1905-018731_PAS_2of2.svs', '18-162_PAS_4of6.svs', 'S-1909-007149_PAS_1of2.svs', 'S-1905-017738_PAS_1of2.svs', 'S-1908-010066_PAS_1of2.svs']
Processing image - S-2103-004857_PAS_2of2
Generating binary masks
Processing image - S-2106-003588_PAS_1of2
Generating binary masks
Processing image - S-2001-005357_PAS_1of2
Generating binary masks
Processing image - S-1904-007293_PAS_1of2
Generating binary masks
performing distance transform
performing distance transform
performing distance transform
performing distance transform
